In [190]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from statsmodels.formula.api import logit
import statsmodels.formula.api as sm
import warnings
warnings.filterwarnings('ignore')

In [191]:
Pro_RB_Path = 'Data/Data_CSVstats_Pro_Football_Ref_RB_Trail.csv'

data = pd.read_csv(Pro_RB_Path)

players = pd.read_csv('Data/Names.csv')

In [192]:
pd.set_option('display.max_columns', None)

In [193]:
data = data.fillna(0)
data = data.drop(columns=["Unnamed: 0", "Pos_"])

In [194]:
def staus(data):
    different_staus_options = []
    temp = data.drop(columns=["Name_", "Unnamed: 2_level_0_Date", 
                              "Unnamed: 6_level_0_Tm", "Unnamed: 7_level_0_Unnamed: 7_level_1",
                              "Unnamed: 9_level_0_Result", "Unnamed: 8_level_0_Opp" ])
    for x in temp.columns:
        statuses = data[x].unique()
        for w in statuses:
            try:
                float(w)
            except:
                try:
                    float(w[:-1])
                except:
                    different_staus_options.append(w)
                    #print(w)
    return different_staus_options

In [195]:
for index, name in enumerate(data["Name_"]):
    if name == 0:
        data["Name_"][index] = data["Name__"][index]
        data["Name__"][index] = 0
data = data.drop(columns = ["Name__"])

In [196]:
stats = staus(data)

In [197]:
set_res = set(stats) 
list_res = (list(set_res))
values_to_replace = []
 
for item in list_res: 
    print(item)
    values_to_replace.append(item)

Did Not Play
Suspended
*
Physically Unable to Perform
Non-Football Injury
Exempt List
Injured Reserve
Inactive
COVID-19 List


In [198]:
for x in values_to_replace:
    data[x] = 0

for w in data.columns:
    for index, z in enumerate(data[w]):
        if values_to_replace.count(z) > 0:
            #print(z, index)
            data[z][index] = 1

In [199]:
for x in values_to_replace:
    data = data.replace(x, 0)

In [200]:
for x in data.columns:
    length = len("Unnamed: ")
    if x[:length] == "Unnamed: ":
        real_col = x.split("_")
        data = data.rename(columns={ x : real_col[-1]})
        #print(real_col[-1])
        

In [201]:
#Delete Duplicate columns
data = data.loc[:,~data.columns.duplicated()].copy()

In [202]:
def result_maker(data):
    data["Win?"] = 0
    data["Lose?"] = 0
    data["Tie?"] = 0
    data["Score-not-counted"] = 0
    data["Tm-Score"] = 0
    data["Opp-Score"] = 0
    for index, x in enumerate(data["Result"]):
        if x[0] == "W" or x[0] == "L" or x[0] == "T":
            if x[0] == "W":
                data["Win?"][index] = 1
            elif x[0] == "L":
                data["Lose?"][index] = 1
            else:
                data["Tie?"][index] = 1
            ind_num = x[1:].split("-")
            if len(ind_num) == 2:
                data["Tm-Score"][index] = ind_num[0]
                data["Opp-Score"][index] = ind_num[1]
        else:
            data["Score-not-counted"][index] = 1

In [203]:
result_maker(data)
data = data.drop(columns=["Result"])

In [204]:
def seperate_date(data):
    data["Day"] = 0
    data["Month"] = 0
    data["No-Date"] = 0
    
    for index, x in enumerate(data["Date"]):
        Year_Month_Day = x.split("-")
        if len(Year_Month_Day) > 1:
            Month = Year_Month_Day[1]
            Day = Year_Month_Day[2]
        
            data["Day"][index] = Day
            data["Month"][index] = Month
        else:
            data["No-Date"][index] = 1
    

In [205]:
seperate_date(data)
data = data.drop(columns=["Date"])

In [206]:
def convert_percentage(data):
    for col in data.columns:
        for index, x in enumerate(data[col]):
            try:
                per = x[-1]
                if per == "%": 
                    data[col] = x[:-1]
            except:
                pass

In [207]:
convert_percentage(data)

In [208]:
def convert_to_number(data):
    temp = data.drop(columns = ["Name_", "Tm", "1", "Opp"])
    didnt_work = {}
    
    for col in temp.columns:
        #print(col)
        for x in data[col]:
            try:
                data[col] = x.astype(float)
            except:
                pass
            
        

In [209]:
convert_to_number(data)

In [210]:
def eliminate_Useless(data, num = 1, drop = False):
    limited = []
    for x in data.columns:
        amount = len(data[x].unique())
        if amount <= num:
            limited.append(x)
            print(x, amount, data[x].unique())
    if drop:
        data = data.drop(columns = limited)

In [211]:
eliminate_Useless(data, num = 5, drop = False)

Def. Snaps_Pct 1 ['0']
Fumbles_TD 4 [0 '0' '1' '0.0']
Off. Snaps_Pct 1 ['12']
Punting_Blck 3 [0. 1. 4.]
Punting_In20 1 [0.]
Punting_In20% 1 ['0.0']
Punting_NY/P 2 [ 0.  31.7]
Punting_Net 2 [ 0. 95.]
Punting_RetYds 1 [0.]
Punting_TB 1 [0.]
Punting_TB% 1 ['0.0']
Receiving_Ctch% 1 ['0.0']
ST Snaps_Pct 1 ['47']
Scoring_FG% 4 [  0.  100.   33.3  28.6]
Scoring_FGA 4 [0. 1. 3. 7.]
Scoring_FGM 3 [0. 1. 2.]
Scoring_Sfty 2 [0 '1']
Scoring_XP% 4 [  0.   75.  100.   90.9]
Scoring_XPA 5 [ 0.  4.  5.  2. 11.]
Scoring_XPM 5 [ 0.  3.  5.  2. 10.]
GS 1 [0]
Sk 5 [0 1.0 '0.0' '1.0' '2.0']
Status 1 [0]
1 2 [0 '@']
Did Not Play 2 [0 1]
Suspended 2 [0 1]
* 2 [0 1]
Physically Unable to Perform 2 [0 1]
Non-Football Injury 2 [0 1]
Exempt List 2 [0 1]
Injured Reserve 2 [0 1]
Inactive 2 [1 0]
COVID-19 List 2 [0 1]
Win? 2 [1 0]
Lose? 2 [0 1]
Tie? 2 [0 1]
Score-not-counted 2 [0 1]
No-Date 2 [0 1]


In [212]:
data = data.drop(columns=["Def. Snaps_Pct",
                          "Off. Snaps_Pct",
                          "Punting_In20", 
                          "Punting_In20%", 
                          "Punting_RetYds",
                          "Punting_TB", 
                          "Punting_TB%",
                          "Receiving_Ctch%", 
                          "ST Snaps_Pct",
                          "GS", 
                          "Sk", 
                          "Status"])

In [213]:
temp = data.drop(columns = {"Name_", "Tm", "1", "Opp"})
for x in temp.columns:
    if temp[x].dtype == object:
        try:
            data[x] = pd.to_numeric(data[x])
        except:
            print(x)

In [214]:
dk_points = pd.read_csv('Data/WeeklyPoints2014-2021.csv', delimiter = ";")
dk_points

,Week,Year,GID,Name,Pos,Team,h/a,Oppt,DK_points,DK_salary
0,17,2014,1426,"Smith, Geno",QB,nyj,a,mia,30.12,5000
1,17,2014,1408,"Griffin III, Robert",QB,was,h,dal,25.34,5700
2,17,2014,1252,"Rodgers, Aaron",QB,gnb,h,det,24.44,8600
3,17,2014,1302,"Flacco, Joe",QB,bal,h,cle,23.58,6200
4,17,2014,1225,"Manning, Eli",QB,nyg,h,phi,23.16,7100
...,...,...,...,...,...,...,...,...,...,...
55003,1,2021,7002,Atlanta,Def,atl,h,phi,0,2000
55004,1,2021,7006,Chicago,Def,chi,a,lar,0,2800
55005,1,2021,7030,Tennessee,Def,ten,h,ari,0,2300
55006,1,2021,7014,Jacksonville,Def,jac,a,hou,-3,2700


In [215]:
title = ['Week', 'Year', 'GID', "DK_points", "DK_salary"]
index_to_remove = []
for name in title:
    for x in range(len(dk_points[name])):
        try:
            dk_points[name][x] = float(dk_points[name][x])
        except ValueError:
            dk_points[name][x] = 0  
            index_to_remove.append(x)
        

In [216]:
# Replace 'DK points' with NaN (or another value of your choice)
dk_points['Week'] = dk_points['Week'].replace('Week', float('nan'))
dk_points['Week'] = dk_points['Week'].astype(float)


dk_points['Year'] = dk_points['Year'].replace('Year', float('nan'))
dk_points['Year'] = dk_points['Year'].astype(float)

dk_points['DK_points'] = dk_points['DK_points'].replace('DK_points', float('nan'))
dk_points['DK_points'] = dk_points['DK_points'].astype(float)

dk_points['DK_salary'] = dk_points['DK_salary'].replace('DK_salary', float('nan'))
dk_points['DK_salary'] = dk_points['DK_salary'].astype(float)

In [217]:
def get_points(data):
    data["Dk_Points"] = 0
    data["Salary"] = 0
    
    for index, row in dk_points.iterrows():
        Name = row["Name"]
        Week = row["Week"]
        Year = row["Year"]
        
        Index = data[(data["Name_"] == Name) & (data["Year"] == float(Year)) & (data["Week"] == int(Week))].index
        
        data["Dk_Points"][Index] = row["DK_points"]
        data["Salary"][Index] = row["DK_salary"]
    return data

In [218]:
get_points(data)

,Def Interceptions_Int,Def Interceptions_PD,Def Interceptions_TD,Def Interceptions_Yds,Def. Snaps_Num,Fumbles_FF,Fumbles_FL,Fumbles_FR,Fumbles_Fmb,Fumbles_TD,Fumbles_Yds,Kick Returns_Rt,Kick Returns_TD,Kick Returns_Y/Rt,Kick Returns_Yds,Name_,Off. Snaps_Num,Passing_AY/A,Passing_Att,Passing_Cmp,Passing_Cmp%,Passing_Int,Passing_Rate,Passing_Sk,Passing_TD,Passing_Y/A,Passing_Yds,Passing_Yds.1,Punt Returns_Ret,Punt Returns_TD,Punt Returns_Y/R,Punt Returns_Yds,Punting_Blck,Punting_NY/P,Punting_Net,Punting_Pnt,Punting_Y/P,Punting_Yds,Receiving_Rec,Receiving_TD,Receiving_Tgt,Receiving_Y/R,Receiving_Y/Tgt,Receiving_Yds,Rushing_Att,Rushing_TD,Rushing_Y/A,Rushing_Yds,ST Snaps_Num,Scoring_2PM,Scoring_FG%,Scoring_FGA,Scoring_FGM,Scoring_Pts,Scoring_Sfty,Scoring_TD,Scoring_XP%,Scoring_XPA,Scoring_XPM,Tackles_Ast,Tackles_Comb,Tackles_QBHits,Tackles_Solo,Tackles_TFL,Rk,Year,G#,Week,Age,Tm,1,Opp,Did Not Play,Suspended,*,Physically Unable to Perform,Non-Football Injury,Exempt List,Injured Reserve,Inactive,COVID-19 List,Win?,Lose?,Tie?,Score-not-counted,Tm-Score,Opp-Score,Day,Month,No-Date,Dk_Points,Salary
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,"Anderson, C.J.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2013.0,0.0,1.0,0.0,DEN,0,BAL,0,0,0,0,0,0,0,1,0,1,0,0,0,49,27,5,9,0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,"Anderson, C.J.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2013.0,0.0,2.0,0.0,DEN,@,NYG,0,0,0,0,0,0,0,1,0,1,0,0,0,41,23,15,9,0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,"Anderson, C.J.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2013.0,0.0,3.0,0.0,DEN,0,OAK,0,0,0,0,0,0,0,1,0,1,0,0,0,37,21,23,9,0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,"Anderson, C.J.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2013.0,0.0,4.0,0.0,DEN,0,PHI,0,0,0,0,0,0,0,1,0,1,0,0,0,52,20,29,9,0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,"Anderson, C.J.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2013.0,0.0,5.0,0.0,DEN,@,DAL,0,0,0,0,0,0,0,1,0,1,0,0,0,51,48,6,10,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38577,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,"Williams, Damien",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,123.0,2022.0,10.0,10.0,0.0,ATL,@,CAR,1,0,0,0,0,0,0,0,0,0,1,0,0,15,25,10,11,0,0.0,0.0
38578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,"Williams, Damien",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,124.0,2022.0,11.0,11.0,0.0,ATL,0,CHI,1,0,0,0,0,0,0,0,0,1,0,0,0,27,24,20,11,0,0.0,0.0
38579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [219]:
#names = pd.DataFrame(data={"Name_":data["Name_"].unique()})
#print(data["Name_"].unique(), "\n", data["Name_"])
#names.to_csv('filtered_data_RB.csv', index=False, sep=";")

In [220]:
def filter_for_Prep(data):
    
    filtered_data = data[data["Year"] >= 2014]
    filtered_data = filtered_data.drop(columns = ["Rk", "G#"])
    filtered_data = filtered_data.loc[:,~data.T.duplicated(keep='first')]
    do_not_drop = ['Injured Reserve','Non-Football Injury','Suspended','Exempt List','Did Not Play','Physically Unable to Perform','Inactive','COVID-19 List','Tie?','Score-not-counted']
    eliminated = []
    eliminated_value = []
    for col in filtered_data.columns:
        num = (filtered_data[col] == 0).sum()/len(filtered_data)
        
        if num > 0.89 and do_not_drop.count(col) == 0:
            eliminated.append(col)
            eliminated_value.append(num)
            filtered_data = filtered_data.drop(columns=[col])
        
        
        
            
    return filtered_data, eliminated

In [221]:
filtered_data, eliminated = filter_for_Prep(data)

In [222]:
heights_and_weights = pd.read_csv('Data/data_RBs.csv', sep = ";")

heights_and_weights["Feet"] = 0
heights_and_weights["Inches"] = 0
heights_and_weights["Weight(lbs)"] = 0

index = 0
for x in heights_and_weights["Info"]:
    feet = x[0]
    inch = x[2]
    weight = x[5:8]
    
    heights_and_weights["Feet"][index] = feet
    heights_and_weights["Inches"][index] = inch
    heights_and_weights["Weight(lbs)"][index] = weight
    index += 1

In [223]:
index = 0
filtered_data = filtered_data.reset_index(drop=True)
filtered_data["Feet"] = 0
filtered_data["Inches"] = 0
filtered_data["Weight(lbs)"] = 0

dumb_players_with_no_weight = []
dumb_players_with_no_weight_index = []
for index, x in enumerate(filtered_data["Name_"]):
    try:
        indices = heights_and_weights[heights_and_weights['Name'] == x].index[0]

        filtered_data["Feet"][index] = heights_and_weights["Feet"][indices]
        filtered_data["Inches"][index] = heights_and_weights["Inches"][indices]
        filtered_data["Weight(lbs)"][index] = heights_and_weights["Weight(lbs)"][indices]
    except:
        dumb_players_with_no_weight.append(x)
        dumb_players_with_no_weight_index.append(index)
        
    index += 1

    
filtered_data.drop(labels = dumb_players_with_no_weight_index, axis = 0)
filtered_data = filtered_data.reset_index(drop=True)

In [243]:
Fan_Points_2013_2022 = pd.read_csv('Data/Fantasy_Points_2013-2022 - Sheet1.csv')
Fan_Points_2022.TEAM = Fan_Points_2022.TEAM.replace("LV", "LVR").replace("JAC", "JAX").replace("SF", "SFO").replace("TB", "TAM").replace("GB", "GNB").replace("KC", "KAN").replace("NO", "NOR").replace("NE", "NWE")

In [244]:
Fan_Points_2013_2022 = Fan_Points_2013_2022[Fan_Points_2013_2022["POS"] == "RB"].reset_index(drop=True)
Fan_Points_2013_2022["Convert_Name"] = 0


for index, name in enumerate(Fan_Points_2013_2022["PLAYER"]):
    name_seg = name.split(" ")
    if len(name_seg) == 2:
        Fan_Points_2013_2022["Convert_Name"][index] = name_seg[1] + ", " + name_seg[0]
    else:
        Fan_Points_2013_2022["Convert_Name"][index] = name_seg[1] + " " + name_seg[2] + ", " + name_seg[0]

In [250]:
def place_2022_scoring(filtered_data, Fan_Points_2013_2022):
    name_options = list(filtered_data["Name_"].unique())
    idoit = []
    accounted = 0
    amount = 0
    for index, row in Fan_Points_2013_2022.iterrows():
        name = row["Convert_Name"]
        week = 1
        if row["Year"] >= 2021:
            Games = row[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18']]
        else:
            Games = row[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17']]
        if name in name_options:
            #print(name)
            for index, col in enumerate(Games.keys(), start=1):
                if col in Games:
                    try:
                        points = float(Games[col])
                    except:
                        points = 0
                        accounted += 1
                        
                    filtered_data.loc[(filtered_data["Year"] == int(row["Year"])) & (filtered_data["Name_"] == name) & (filtered_data["Week"] == week), 'Dk_Points'] = points
                amount += 1
                week += 1
        else:
            idoit.append(name)
    print(len(idoit)/len(list(filtered_data["Name_"].unique())) * 100, "% of the points sheet are not in", "\n", accounted/amount * 100, "% was accounted for")
    filtered_data["Dk_Points"] = filtered_data["Dk_Points"].astype(float)
    return idoit

In [251]:
idoit = place_2022_scoring(filtered_data, Fan_Points_2013_2022)
idoit

37.659033078880405 % of the points sheet are not in 
 35.34225224055072 % was accounted for


['Corbin, Jashaun',
 'Koback, Bryant',
 'Lovett, John',
 'Mason, Ben',
 'Platzgummer, Sandro',
 'Barber, Peyton',
 'White, James',
 'Peterson, Adrian',
 'Richard, Jalen',
 'Gallman Jr., Wayne',
 'Smith, Rodney',
 'Armah Jr., Alex',
 'Stanton IV, Johnny',
 'Hardy, JaQuan',
 'Huntley, Jason',
 'Phillips, Scottie',
 'Kelly Jr., John',
 'Patrick, Jacques',
 'Howell, Buddy',
 'Hill, Brian',
 'Ralston, Nick',
 'Carter, Tory',
 'Olonilua, Sewo',
 'Johnson, Josh',
 'Doaks, Gerrid',
 'Emmons, B.J.',
 'Thompson, Darwin',
 'Platzgummer, Sandro',
 'Williams Jr., Pooka',
 'Gallman Jr., Wayne',
 'Peterson, Adrian',
 'White, James',
 'Hill, Brian',
 'Johnson, Kerryon',
 'Barber, Peyton',
 'Richard, Jalen',
 'Thompson, Chris',
 'Adams, Josh',
 'Smith, Rodney',
 'Thompson, Darwin',
 'McCoy, LeSean',
 'Williams, Antonio',
 'Howell, Buddy',
 'Armah Jr., Alex',
 'Scarbrough, Bo',
 'Cox, Chandler',
 'Huntley, Jason',
 'Phillips, Scottie',
 'Miller, Lamar',
 'Foster, D.J.',
 'Williams, Dexter',
 'Brooks-Jam

In [189]:
# more_forgotten_RB = {
#     "Jacobs, Josh" : "https://www.pro-football-reference.com/players/J/JacoJo01/gamelog/",
#     "Williams, Jamaal" : "https://www.pro-football-reference.com/players/W/WillJa06/gamelog/",
#     "Sanders, Miles" : "https://www.pro-football-reference.com/players/S/SandMi01/gamelog/",
#     "Swift, D'Andre" : "https://www.pro-football-reference.com/players/S/SwifDA00/gamelog/",
#     "Pierce, Dameon" : "https://www.pro-football-reference.com/players/P/PierDa01/gamelog/",
#     "Taylor, Jonathan" : "https://www.pro-football-reference.com/players/T/TaylJo02/gamelog/",
#     "White, Rachaad" : "https://www.pro-football-reference.com/players/W/WhitRa01/gamelog/", 
#     "Foreman, D'Onta" : "https://www.pro-football-reference.com/players/F/ForeDO00/gamelog/"
#     "Carter, Michael" : "https://www.pro-football-reference.com/players/C/CartMi03/gamelog/",
#     "Hall, Breece" : "https://www.pro-football-reference.com/players/H/HallBr03/gamelog/",
#     "Robinson Jr., Brian" : "https://www.pro-football-reference.com/players/R/RobiBr01/gamelog/",
#     "Cook, James" : "https://www.pro-football-reference.com/players/C/CookJa01/gamelog/",
#     "Harris, Damien" : "https://www.pro-football-reference.com/players/H/HarrDa06/gamelog/",
#     "Mattison, Alexander" : "https://www.pro-football-reference.com/players/M/MattAl01/gamelog/",
#     "Dobbins, J.K." : "https://www.pro-football-reference.com/players/D/DobbJK00/gamelog/",
#     "Hasty, JaMycal" : "https://www.pro-football-reference.com/players/H/HastJa02/gamelog/",
#     "Ingram II, Mark" : "https://www.pro-football-reference.com/players/I/IngrMa01/gamelog/"
#     "Scott, Boston" : "https://www.pro-football-reference.com/players/S/ScotBo02/gamelog/",
#     "Williams, Javonte" : "https://www.pro-football-reference.com/players/W/WillJa10/gamelog/",
#     "Ham, C.J." : "https://www.pro-football-reference.com/players/H/HamxC.00/gamelog/",
#     "Williams, Kyren" : "https://www.pro-football-reference.com/players/W/WillKy02/gamelog/",
#     "Williams, Jonathan" : "https://www.pro-football-reference.com/players/W/WillJo07/gamelog/",
#     "Coleman, Tevin" : "https://www.pro-football-reference.com/players/C/ColeTe01/gamelog/",
#     "Williams, Darrel" : "https://www.pro-football-reference.com/players/W/WillDa10/gamelog/",
#     "Jones II, Ronald" : "https://www.pro-football-reference.com/players/J/JoneRo01/gamelog/",
#     "Evans, Chris" : "https://www.pro-football-reference.com/players/E/EvanCh02/gamelog/",
#     "Harris, Kevin" : "https://www.pro-football-reference.com/players/H/HarrKe01/gamelog/",
#     "Jones Jr., Tony" : "https://www.pro-football-reference.com/players/J/JoneTo04/gamelog/",
#     "Montgomery II, Ty" : "https://www.pro-football-reference.com/players/M/MontTy01/gamelog/",
#     "Rivers, Ronnie" : "https://www.pro-football-reference.com/players/R/RiveRo01/gamelog/",
#     "Davis-Price, Tyrion" : "https://www.pro-football-reference.com/players/D/DaviTy03/gamelog/",
#     "Johnson, David" : "https://www.pro-football-reference.com/players/J/JohnDa08/gamelog/",
#     "Williams, Trayveon" : "https://www.pro-football-reference.com/players/W/WillTr06/gamelog/",
#     "White, Zamir" : "https://www.pro-football-reference.com/players/W/WhitZa01/gamelog/",
#     "Johnson, Jakob" : "https://www.pro-football-reference.com/players/J/JohnJa12/gamelog/",
#     "Johnson, D'Ernest" : "https://www.pro-football-reference.com/players/J/JohnDE04/gamelog/",
#     "Taylor, J.J." : "https://www.pro-football-reference.com/players/T/TaylJJ01/gamelog/",
#     "Smith, Keith" : "https://www.pro-football-reference.com/players/S/SmitKe03/gamelog/",
#     "Williams, Damien" : "https://www.pro-football-reference.com/players/W/WillDa05/gamelog/"
    
# }

In [147]:
#last = []
#first = []

# for x in idoit:
#     sep = x.split(", ")
#     last.append(sep[0])
#     first.append(sep[1])

In [148]:
#forgotten = pd.DataFrame(data={"Name_":idoit, "First Name": first, "Last Name": last})
#print(data["Name_"].unique(), "\n", data["Name_"])
#forgotten.to_csv('Forgotten_RB.csv', index=False, sep=";")

In [228]:
Offesive_Team_Stats = pd.read_csv('Data/NFL Team Season Stats - Offense Reg Season.csv', thousands=',')
Defensive_Team_Stats = pd.read_csv('Data/NFL Team Season Stats - Defense Reg Season.csv', thousands=',')
filtered_data["Tm"] = filtered_data["Tm"].replace("STL", "NOR").replace("OAK", "LVR").replace("SDG", "LAC")

In [229]:
def offensive_scoring_creation(filtered_data):
    filtered_data["Last_Off_Total_YDS"] = 0
    filtered_data["Last_Off_Total_YDS/G"] = 0
    filtered_data["Last_Off_Passing_YDS"] = 0
    filtered_data["Last_Off_Passing_YDS/G"] = 0
    filtered_data["Last_Off_Rushing_YDS"] = 0
    filtered_data["Last_Off_Rushing_YDS/G"] = 0
    filtered_data["Last_Off_Points_PTS"] = 0
    filtered_data["Last_Off_PointsPTS/G"] = 0
    for x in filtered_data.index:
        Team = filtered_data.loc[x]["Tm"]
        Year = filtered_data.loc[x]["Year"]
        row = Offesive_Team_Stats[(Offesive_Team_Stats["TEAM"] == Team) & (Offesive_Team_Stats["Year"] == Year)]

        # Ensure the conditions return only one row
        if len(row) == 1:
            filtered_data.loc[x, "Last_Off_Total_YDS"] = row["Total_YDS"].iloc[0]
            filtered_data.loc[x, "Last_Off_Total_YDS/G"] = row["Total_YDS/G"].iloc[0]
            filtered_data.loc[x, "Last_Off_Passing_YDS"] = row["Passing_YDS"].iloc[0]
            filtered_data.loc[x, "Last_Off_Passing_YDS/G"] = row["Passing_YDS/G"].iloc[0]
            filtered_data.loc[x, "Last_Off_Rushing_YDS"] = row["Rushing_YDS"].iloc[0]
            filtered_data.loc[x, "Last_Off_Rushing_YDS/G"] = row["Rushing_YDS/G"].iloc[0]
            filtered_data.loc[x, "Last_Off_Points_PTS"] = row["Points_PTS"].iloc[0]
            filtered_data.loc[x, "Last_Off_PointsPTS/G"] = row["PointsPTS/G"].iloc[0]

In [230]:
def def_scoring_creation(filtered_data):
    filtered_data["Last_Def_Total_YDS"] = 0
    filtered_data["Last_Def_Total_YDS/G"] = 0
    filtered_data["Last_Def_Passing_YDS"] = 0
    filtered_data["Last_Def_Passing_YDS/G"] = 0
    filtered_data["Last_Def_Rushing_YDS"] = 0
    filtered_data["Last_Def_Rushing_YDS/G"] = 0
    filtered_data["Last_Def_Points_PTS"] = 0
    filtered_data["Last_Def_PointsPTS/G"] = 0
    for x in filtered_data.index:
        Team = filtered_data.loc[x]["Tm"]
        Year = filtered_data.loc[x]["Year"]
        row = Defensive_Team_Stats[(Defensive_Team_Stats["TEAM"] == Team) & (Defensive_Team_Stats["Year"] == Year)]

        # Ensure the conditions return only one row
        if len(row) == 1:
            filtered_data.loc[x, "Last_Def_Total_YDS"] = row["Total_YDS"].iloc[0]
            filtered_data.loc[x, "Last_Def_Total_YDS/G"] = row["Total_YDS/G"].iloc[0]
            filtered_data.loc[x, "Last_Def_Passing_YDS"] = row["Passing_YDS"].iloc[0]
            filtered_data.loc[x, "Last_Def_Passing_YDS/G"] = row["Passing_YDS/G"].iloc[0]
            filtered_data.loc[x, "Last_Def_Rushing_YDS"] = row["Rushing_YDS"].iloc[0]
            filtered_data.loc[x, "Last_Def_Rushing_YDS/G"] = row["Rushing_YDS/G"].iloc[0]
            filtered_data.loc[x, "Last_Def_Points_PTS"] = row["Points_PTS"].iloc[0]
            filtered_data.loc[x, "Last_Def_PointsPTS/G"] = row["PointsPTS/G"].iloc[0]

In [252]:
#delete duplicate rows
filtered_data = filtered_data.drop_duplicates(keep='last')

In [232]:
def_scoring_creation(filtered_data)
offensive_scoring_creation(filtered_data)

In [233]:
def wr_week_one(data, team, opp, players, year):
    
    summary_team_stats = pd.DataFrame(data={"Last_Off_Total_YDS" : [0], 
                                            "Last_Off_Total_YDS/G" : [0], 
                                            "Last_Off_Passing_YDS" : [0],
                                            "Last_Off_Passing_YDS/G" : [0], 
                                            "Last_Off_Rushing_YDS" : [0], 
                                            "Last_Off_Rushing_YDS/G" : [0], 
                                            "Last_Off_Points_PTS" : [0], 
                                            "Last_Off_PointsPTS/G" : [0], 
                                            "Last_Def_Total_YDS" : [0],
                                            "Last_Def_Total_YDS/G" : [0], 
                                            "Last_Def_Passing_YDS" : [0], 
                                            "Last_Def_Passing_YDS/G" : [0], 
                                            "Last_Def_Rushing_YDS" : [0], 
                                            "Last_Def_Rushing_YDS/G" : [0], 
                                            "Last_Def_Points_PTS" : [0],
                                            "Last_Def_PointsPTS/G" : [0]})
    
    temp = pd.DataFrame(data = {"Name_":players})
    drop = ["Name_", "Tm", "1", "Opp", "Week", "Year", "Day", 
            'Tie?','Score-not-counted', "Weight(lbs)", "Inches", "Feet",
           "Last_Off_Total_YDS", "Last_Off_Total_YDS/G", "Last_Off_Passing_YDS",
           "Last_Off_Passing_YDS/G", "Last_Off_Rushing_YDS", "Last_Off_Rushing_YDS/G", 
           "Last_Off_Points_PTS", "Last_Off_PointsPTS/G", "Last_Def_Total_YDS",
           "Last_Def_Total_YDS/G", "Last_Def_Passing_YDS", "Last_Def_Passing_YDS/G", 
           "Last_Def_Rushing_YDS", "Last_Def_Rushing_YDS/G", "Last_Def_Points_PTS",
           "Last_Def_PointsPTS/G"]
    #temp = data.drop(columns = ["Name_", "Tm", "1", "Opp"])
    used_data = data.drop(columns = drop)
    
    for col in used_data.columns:
            if used_data[col].dtype != object:
                #temp[col + "_Count"] = 0
                temp[col + "_Average"] = 0
                temp[col + "_STD"] = 0
                temp[col + "_Min"] = 0
                temp[col + "_25%"] = 0
                temp[col + "_50%"] = 0
                temp[col + "_100%"] = 0
                temp[col + "_Max"] = 0
                
                
                #temp["History_Opp_" + col + "_Count"] = 0
                temp["History_" + col + "_Average"] = 0
                temp["History_" + col + "_STD"] = 0
                temp["History_" + col + "_Min"] = 0
                temp["History_" + col + "_25%"] = 0
                temp["History_" + col + "_50%"] = 0
                temp["History_" + col + "_100%"] = 0
                temp["History_" + col + "_Max"] = 0
    
    
    
    seperate_df = list(summary_team_stats.columns)
    
    temp["Feet"] = 0
    temp["Inches"] = 0
    temp["Weight(lbs)"] = 0
    temp["Actaul_Dk_Points"] = 0    
                
    sub_data = data[data["Year"] > 2013]
    #History of all games per player on team   
    index = 0
    for name in players:
        indiv_player = data[(data["Name_"] == name) & (data["Year"] < year)]
        indiv_player = indiv_player.drop(columns = drop)
       
        
        for col in indiv_player:
            if seperate_df.count(col) == 0:
                stats = indiv_player[col].describe()
                #temp[col + "_Count"][index] = stats[0]
                temp[col + "_Average"][index] = stats[1]
                temp[col + "_STD"][index] = stats[2]
                temp[col + "_Min"][index] = stats[3]
                temp[col + "_25%"][index] = stats[4]
                temp[col + "_50%"][index] = stats[5]
                temp[col + "_100%"][index] = stats[6]
                temp[col + "_Max"][index] = stats[7]   
        index += 1
    
    
    #History of all players on team vs oppenant
    index = 0 
    for name in players:
        last_games = data[(data["Name_"] == name) & (data["Opp"] == opp) & (data["Year"] < year)]
        last_games = last_games.drop(columns = drop)
        
        for col in last_games:
            if seperate_df.count(col) == 0:
                stats = last_games[col].describe()
                #temp["History_Opp_" + col + "_Count"][index] = stats[0]
                temp["History_" + col + "_Average"][index] = stats[1]
                temp["History_" + col + "_STD"][index] = stats[2]
                temp["History_" + col + "_Min"][index] = stats[3]
                temp["History_" + col + "_25%"][index] = stats[4]
                temp["History_" + col + "_50%"][index] = stats[5]
                temp["History_" + col + "_100%"][index] = stats[6]
                temp["History_" + col + "_Max"][index] = stats[7]   
        index += 1
    
    temp["Opp"] = opp
    temp["Team"] = team
    
    
    
    
    #Get the real points they made and bio
    index = 0
    for name in players:
        
        test_game = data[(data["Name_"] == name) & (data["Opp"] == opp) & (data["Year"] == year) & (data["Week"] == 1)]

        # Assuming test_game contains only one row, otherwise this will error out.
        if len(test_game) == 1:
            #print(test_game["Dk_Points"].iloc[0])

            temp.loc[index, "Actaul_Dk_Points"] = test_game["Dk_Points"].iloc[0]
            temp.loc[index, "Feet"] = test_game["Feet"].iloc[0]
            temp.loc[index, "Inches"] = test_game["Inches"].iloc[0]
            temp.loc[index, "Weight(lbs)"] = test_game["Weight(lbs)"].iloc[0]
        else:
            # Handle cases where test_game might have more than one row or no rows.
            print(f"Expected only one row for conditions, but found {len(test_game)} rows.")
        
        index += 1
    
    summary_team_stats["Last_Off_Total_YDS"] = float(test_game["Last_Off_Total_YDS"])
    summary_team_stats["Last_Off_Total_YDS/G"] = float(test_game["Last_Off_Total_YDS"])
    summary_team_stats["Last_Off_Passing_YDS"] = float(test_game["Last_Off_Passing_YDS"])
    summary_team_stats["Last_Off_Passing_YDS/G"] = float(test_game["Last_Off_Passing_YDS/G"])
    summary_team_stats["Last_Off_Rushing_YDS"] = float(test_game["Last_Off_Rushing_YDS"])
    summary_team_stats["Last_Off_Rushing_YDS/G"] = float(test_game["Last_Off_Rushing_YDS/G"])
    summary_team_stats["Last_Off_Points_PTS"] = float(test_game["Last_Off_Points_PTS"])
    summary_team_stats["Last_Off_PointsPTS/G"] = float(test_game["Last_Off_PointsPTS/G"])
    
    summary_team_stats["Last_Def_Total_YDS"] = float(test_game["Last_Def_Total_YDS"])
    summary_team_stats["Last_Def_Total_YDS/G"] = float(test_game["Last_Def_Total_YDS/G"])
    summary_team_stats["Last_Def_Passing_YDS"] = float(test_game["Last_Def_Passing_YDS"])
    summary_team_stats["Last_Def_Passing_YDS/G"] = float(test_game["Last_Def_Passing_YDS/G"])
    summary_team_stats["Last_Def_Rushing_YDS"] = float(test_game["Last_Def_Rushing_YDS"])
    summary_team_stats["Last_Def_Rushing_YDS/G"] = float(test_game["Last_Def_Rushing_YDS/G"])
    summary_team_stats["Last_Def_Points_PTS"] = float(test_game["Last_Def_Points_PTS"])
    summary_team_stats["Last_Def_PointsPTS/G"] = float(test_game["Last_Def_PointsPTS/G"])
        
        
        
        
        
    
    temp = temp.fillna(0)
    
    
    return temp, summary_team_stats

In [253]:
ready_data, team_data = wr_week_one(filtered_data, "CIN", "PIT", ["Perine, Samaje", "Mixon, Joe"], year = 2022)

In [254]:
ready_data

,Name_,Off. Snaps_Num_Average,Off. Snaps_Num_STD,Off. Snaps_Num_Min,Off. Snaps_Num_25%,Off. Snaps_Num_50%,Off. Snaps_Num_100%,Off. Snaps_Num_Max,History_Off. Snaps_Num_Average,History_Off. Snaps_Num_STD,History_Off. Snaps_Num_Min,History_Off. Snaps_Num_25%,History_Off. Snaps_Num_50%,History_Off. Snaps_Num_100%,History_Off. Snaps_Num_Max,Receiving_Rec_Average,Receiving_Rec_STD,Receiving_Rec_Min,Receiving_Rec_25%,Receiving_Rec_50%,Receiving_Rec_100%,Receiving_Rec_Max,History_Receiving_Rec_Average,History_Receiving_Rec_STD,History_Receiving_Rec_Min,History_Receiving_Rec_25%,History_Receiving_Rec_50%,History_Receiving_Rec_100%,History_Receiving_Rec_Max,Receiving_Tgt_Average,Receiving_Tgt_STD,Receiving_Tgt_Min,Receiving_Tgt_25%,Receiving_Tgt_50%,Receiving_Tgt_100%,Receiving_Tgt_Max,History_Receiving_Tgt_Average,History_Receiving_Tgt_STD,History_Receiving_Tgt_Min,History_Receiving_Tgt_25%,History_Receiving_Tgt_50%,History_Receiving_Tgt_100%,History_Receiving_Tgt_Max,Receiving_Y/R_Average,Receiving_Y/R_STD,Receiving_Y/R_Min,Receiving_Y/R_25%,Receiving_Y/R_50%,Receiving_Y/R_100%,Receiving_Y/R_Max,History_Receiving_Y/R_Average,History_Receiving_Y/R_STD,History_Receiving_Y/R_Min,History_Receiving_Y/R_25%,History_Receiving_Y/R_50%,History_Receiving_Y/R_100%,History_Receiving_Y/R_Max,Receiving_Y/Tgt_Average,Receiving_Y/Tgt_STD,Receiving_Y/Tgt_Min,Receiving_Y/Tgt_25%,Receiving_Y/Tgt_50%,Receiving_Y/Tgt_100%,Receiving_Y/Tgt_Max,History_Receiving_Y/Tgt_Average,History_Receiving_Y/Tgt_STD,History_Receiving_Y/Tgt_Min,History_Receiving_Y/Tgt_25%,History_Receiving_Y/Tgt_50%,History_Receiving_Y/Tgt_100%,History_Receiving_Y/Tgt_Max,Receiving_Yds_Average,Receiving_Yds_STD,Receiving_Yds_Min,Receiving_Yds_25%,Receiving_Yds_50%,Receiving_Yds_100%,Receiving_Yds_Max,History_Receiving_Yds_Average,History_Receiving_Yds_STD,History_Receiving_Yds_Min,History_Receiving_Yds_25%,History_Receiving_Yds_50%,History_Receiving_Yds_100%,History_Receiving_Yds_Max,Rushing_Att_Average,Rushing_Att_STD,Rushing_Att_Min,Rushing_Att_25%,Rushing_Att_50%,Rushing_Att_100%,Rushing_Att_Max,History_Rushing_Att_Average,History_Rushing_Att_STD,History_Rushing_Att_Min,History_Rushing_Att_25%,History_Rushing_Att_50%,History_Rushing_Att_100%,History_Rushing_Att_Max,Rushing_TD_Average,Rushing_TD_STD,Rushing_TD_Min,Rushing_TD_25%,Rushing_TD_50%,Rushing_TD_100%,Rushing_TD_Max,History_Rushing_TD_Average,History_Rushing_TD_STD,History_Rushing_TD_Min,History_Rushing_TD_25%,History_Rushing_TD_50%,History_Rushing_TD_100%,History_Rushing_TD_Max,Rushing_Y/A_Average,Rushing_Y/A_STD,Rushing_Y/A_Min,Rushing_Y/A_25%,Rushing_Y/A_50%,Rushing_Y/A_100%,Rushing_Y/A_Max,History_Rushing_Y/A_Average,History_Rushing_Y/A_STD,History_Rushing_Y/A_Min,History_Rushing_Y/A_25%,History_Rushing_Y/A_50%,History_Rushing_Y/A_100%,History_Rushing_Y/A_Max,Rushing_Yds_Average,Rushing_Yds_STD,Rushing_Yds_Min,Rushing_Yds_25%,Rushing_Yds_50%,Rushing_Yds_100%,Rushing_Yds_Max,History_Rushing_Yds_Average,History_Rushing_Yds_STD,History_Rushing_Yds_Min,History_Rushing_Yds_25%,History_Rushing_Yds_50%,History_Rushing_Yds_100%,History_Rushing_Yds_Max,ST Snaps_Num_Average,ST Snaps_Num_STD,ST Snaps_Num_Min,ST Snaps_Num_25%,ST Snaps_Num_50%,ST Snaps_Num_100%,ST Snaps_Num_Max,History_ST Snaps_Num_Average,History_ST Snaps_Num_STD,History_ST Snaps_Num_Min,History_ST Snaps_Num_25%,History_ST Snaps_Num_50%,History_ST Snaps_Num_100%,History_ST Snaps_Num_Max,Scoring_Pts_Average,Scoring_Pts_STD,Scoring_Pts_Min,Scoring_Pts_25%,Scoring_Pts_50%,Scoring_Pts_100%,Scoring_Pts_Max,History_Scoring_Pts_Average,History_Scoring_Pts_STD,History_Scoring_Pts_Min,History_Scoring_Pts_25%,History_Scoring_Pts_50%,History_Scoring_Pts_100%,History_Scoring_Pts_Max,Scoring_TD_Average,Scoring_TD_STD,Scoring_TD_Min,Scoring_TD_25%,Scoring_TD_50%,Scoring_TD_100%,Scoring_TD_Max,History_Scoring_TD_Average,History_Scoring_TD_STD,History_Scoring_TD_Min,History_Scoring_TD_25%,History_Scoring_TD_50%,History_Scoring_TD_100%,History_Scoring_TD_Max,Age_Average,Age_STD,Age_Min

In [255]:
team_data

,Last_Off_Total_YDS,Last_Off_Total_YDS/G,Last_Off_Passing_YDS,Last_Off_Passing_YDS/G,Last_Off_Rushing_YDS,Last_Off_Rushing_YDS/G,Last_Off_Points_PTS,Last_Off_PointsPTS/G,Last_Def_Total_YDS,Last_Def_Total_YDS/G,Last_Def_Passing_YDS,Last_Def_Passing_YDS/G,Last_Def_Rushing_YDS,Last_Def_Rushing_YDS/G,Last_Def_Points_PTS,Last_Def_PointsPTS/G
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [238]:
filtered_data[(filtered_data["Name_"] == "Perine, Samaje") & (filtered_data["Year"] == 2022)]

,Name_,Off. Snaps_Num,Receiving_Rec,Receiving_Tgt,Receiving_Y/R,Receiving_Y/Tgt,Receiving_Yds,Rushing_Att,Rushing_TD,Rushing_Y/A,Rushing_Yds,ST Snaps_Num,Scoring_Pts,Scoring_TD,Year,Week,Age,Tm,1,Opp,Did Not Play,Suspended,*,Physically Unable to Perform,Non-Football Injury,Exempt List,Injured Reserve,Inactive,COVID-19 List,Win?,Lose?,Tie?,Score-not-counted,Tm-Score,Opp-Score,Day,Month,Dk_Points,Salary,Feet,Inches,Weight(lbs),Last_Def_Total_YDS,Last_Def_Total_YDS/G,Last_Def_Passing_YDS,Last_Def_Passing_YDS/G,Last_Def_Rushing_YDS,Last_Def_Rushing_YDS/G,Last_Def_Points_PTS,Last_Def_PointsPTS/G,Last_Off_Total_YDS,Last_Off_Total_YDS/G,Last_Off_Passing_YDS,Last_Off_Passing_YDS/G,Last_Off_Rushing_YDS,Last_Off_Rushing_YDS/G,Last_Off_Points_PTS,Last_Off_PointsPTS/G
10967,"Perine, Samaje",26.0,3.0,5.0,11.00,6.60,33.0,1.0,0.0,4.00,4.0,21.0,0.0,0.0,2022.0,1.0,26.360,CIN,0,PIT,0,0,0,0,0,0,0,0,0,0,1,0,0,20,23,11,9,6.7,0.0,5,1,24,5613,330.2,3415,200.9,2198,129.3,342,20.1,6034,354.9,3736,219.8,2298,135.2,467,27.5
10968,"Perine, Samaje",14.0,1.0,1.0,8.00,8.00,8.0,1.0,0.0,2.00,2.0,18.0,0.0,0.0,2022.0,2.0,27.002,CIN,@,DAL,0,0,0,0,0,0,0,0,0,0,1,0,0,17,20,18,9,2.0,0.0,5,1,24,5613,330.2,3415,200.9,2198,129.3,342,20.1,6034,354.9,3736,219.8,2298,135.2,467,27.5
10969,"Perine, Samaje",23.0,2.0,2.0,7.00,7.00,14.0,9.0,0.0,5.22,47.0,16.0,6.0,1.0,2022.0,3.0,27.009,CIN,@,NYJ,0,0,0,0,0,0,0,0,0,1,0,0,0,27,12,25,9,14.1,0.0,5,1,24,5613,330.2,3415,200.9,2198,129.3,342,20.1,6034,354.9,3736,219.8,2298,135.2,467,27.5
10970,"Perine, Samaje",12.0,0.0,1.0,0.00,0.00,0.0,1.0,0.0,0.00,0.0,18.0,0.0,0.0,2022.0,4.0,27.013,CIN,0,MIA,0,0,0,0,0,0,0,0,0,1,0,0,0,27,15,29,9,0.0,0.0,5,1,24,5613,330.2,3415,200.9,2198,129.3,342,20.1,6034,354.9,3736,219.8,2298,135.2,467,27.5
10971,"Perine, Samaje",19.0,4.0,4.0,9.75,9.75,39.0,3.0,0.0,5.67,17.0,15.0,0.0,0.0,2022.0,5.0,27.023,CIN,@,BAL,0,0,0,0,0,0,0,0,0,0,1,0,0,17,19,9,10,9.6,0.0,5,1,24,5613,330.2,3415,200.9,2198,129.3,342,20.1,6034,354.9,3736,219.8,2298,135.2,467,27.5
10972,"Perine, Samaje",14.0,2.0,2.0,5.50,5.50,11.0,2.0,0.0,2.50,5.0,18.0,0.0,0.0,2022.0,6.0,27.030,CIN,@,NOR,0,0,0,0,0,0,0,0,0,1,0,0,0,30,26,16,10,3.6,0.0,5,1,24,5613,330.2,3415,200.9,2198,129.3,342,20.1,6034,354.9,3736,219.8,2298,135.2,467,27.5
10973,"Perine, Samaje",20.0,2.0,2.0,4.00,4.00,8.0,1.0,0.0,0.00,0.0,13.0,0.0,0.0,2022.0,7.0,27.037,CIN,0,ATL,0,0,0,0,0,0,0,0,0,1,0,0,0,35,17,23,10,2.8,0.0,5,1,24,5613,330.2,3415,200.9,2198,129.3,342,20.1,6034,354.9,3736,219.8,2298,135.2,467,27.5
10974,"Perine, Samaje",13.0,3.0,3.0,3.33,3.33,10.0,1.0,0.0,7.00,7.0,13.0,0.0,0.0,2022.0,8.0,27.045,CIN,@,CLE,0,0,0,0,0,0,0,0,0,0,1,0,0,13,32,31,10,4.7,0.0,5,1,24,5613,330.2,3415,200.9,2198,129.3,342,20.1,6034,354.9,3736,219.8,2298,135.2,467,27.5
10975,"Perine, Samaje",27.0,1.0,2.0,3.00,1.50,3.0,6.0,0.0,8.50,51.0,16.0,0.0,0.0,2022.0,9.0,27.051,CIN,0,CAR,0,0,0,0,0,0,0,0,0,1,0,0,0,42,21,6,11,6.4,0.0,5,1,24,5613,330.2,3415,200.9,2198,129.3,342,20.1,6034,354.9,3736,219.8,2298,135.2,467,27.5
10976,"Perine, Samaje",46.0,4.0,4.0,13.00,13.00,52.0,11.0,0.0,2.73,30.0,9.0,18.0,3.0,2022.0,11.0,27.065,CIN,@,PIT,0,0,0,0,0,0,0,0,0,1,0,0,0,37,30,20,11,30.2,0.0,5,1,24,5613,330.2,3415,200.9,2198,129.3,342,20.1,6034,354.9,3736,219.8,2298,135.2,467,27.5
